<a href="https://colab.research.google.com/github/PavanDaniele/drone-person-detection/blob/main/model_Training_and_Evaluation_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up: mount drive + import libraries

**Important Information:** We need to activate the GPU on Colab (_Runtime --> Change runtime type_). \
Every time you start a new session (or reopen the notebook after a few hours) check that the GPU is still active. If we are not using the GPU it can take up to tens of hours to train the models. \
_GPU T4 is the best choice._

In [1]:
# Run this Every time you start a new session
from google.colab import drive
drive.mount('/content/drive') # to mount google drive (to see/access it)

Mounted at /content/drive


In [ ]:
!pip install ultralytics # Installation of Ultralytics for YOLO models

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from ultralytics import YOLO # Import of Ultralytics for YOLO models

In [3]:
import shutil
import os
from PIL import Image

# Import base for EfficientDet:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchvision import transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

from effdet.efficientdet import HeadNet
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchPredict
from effdet.bench import unwrap_bench
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
from torchvision import transforms
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# General Explanation

### Backbone

In Computer Vision, a _Backbone_ is the part of a convolutional neural network responsible for extracting the main features from an image. \
It serves as the shared base upon which subsequent modules are built (such as heads for classification, object detection, segmentation, etc.).

\
Each backbone has been pre-trained on specific datasets (e.g., ImageNet) using particular preprocessing steps, input dimensions, normalization, and augmentation techniques, which should ideally be replicated during fine-tuning to maintain compatibility and achieve optimal performance.

### Data Loader

To train a deep learning model, it is essential to properly handle data loading and preparation. This is the task of the _Data Loader_, a component responsible for:
- Loading images and their corresponding annotations (e.g., .txt or .json) from the dataset.
- Applying preprocessing operations, such as resizing, normalization, data augmentation, etc.
- Organizing data into batches to feed the model during training.

\
Considering the limited resources of my development environment, at first I decided to perform the image and annotation resizing in a separate phase (prior to training), in order to:
- Reduce the workload of the data loader during fine-tuning;
- Increase data loading and training speed;
- Ensure consistency between images and annotations.

But due to the different type of scaling technique, I want to try to fine-tuning the model without any pre-scaling.

The other transformations, instead, are handled by the model-specific data loader, since each model uses different preprocessing and normalization techniques. \
Moreover, some models require specific transformations to achieve optimal performance, and the libraries that provide the models (e.g., Ultralytics for YOLO, torchvision for EfficientDet/SSD) already implement loaders that are properly configured and optimized.

### Normalization

Image normalization consists in scaling pixel values from the range [0, 255] to a more suitable interval (e.g., [0, 1] or [-1, 1]), often based on the mean and standard deviation of the pre-training dataset, with the goal of:
- Avoiding overly large values in the tensors;
- Making the model more stable during training;
- Speeding up convergence.

\
Normalization helps maintain a consistent pixel range and distribution, which is essential for pre-trained models.

### Data Augmentations

Data augmentation consists of random transformations (e.g., rotations, flips, crops, brightness changes, etc.) applied during training. Their purpose is to:
- Simulate new visual conditions;
- Increase dataset variety;
- Reduce overfitting by improving the model’s ability to generalize.

\
In practice, the semantic content of the image doesn't change (e.g., a person remains a person), but its visual appearance is altered to help the model "learn better."


My goal is to evaluate the real-world performance of each model in its ideal scenario, in order to select the most suitable one for deployment on the Jetson Nano. \
For this reason, each model is trained using its native augmentations, meaning the ones that were designed and optimized as part of its original architecture. \
It wouldn’t make sense to disable them or enforce a uniform setup across models, because what we want to observe is the maximum potential of each model, working in the way it was designed to perform best.

# Fine-Tuning Model

### YOLOv8n

First of all we need to save the dataset locally:


In [ ]:
src = '/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n'
dst = '/content/AERALIS_YOLOv8n_local'  # is now on the local VM, NOT on drive

# If the destination folder already exists, I delete it
if os.path.exists(dst):
  shutil.rmtree(dst)

# Recursive copy of ENTIRE folder (and subfolders)
shutil.copytree(src, dst)
print("Copy completed:", os.path.exists(dst))

Copy completed: True


Let's check the total free space:

In [ ]:
!df -h / # It shows the total, used and free space on the root (/) of the Colab VM.

# Avail column: space still available for your files.

Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   56G   58G  49% /


In [ ]:
# Show space used by your local folder
!du -sh /content/AERALIS_YOLOv8n_local

6.6G	/content/AERALIS_YOLOv8n_local


In [ ]:
# Show space occupied by various folders in /content/.
!du -h --max-depth=1 /content/

140K	/content/.config
du: cannot access '/content/drive/.Encrypted/.shortcut-targets-by-id/1LQbD7p_iS5KLqGNdfrYEvsAx0i_bgB0h/projectUPV': No such file or directory
67G	/content/drive
6.6G	/content/AERALIS_YOLOv8n_local
31M	/content/runs_finetune
55M	/content/sample_data
74G	/content/


We want to create the data.yaml file, which YOLO uses to know:
- the path to the training, validation, and test images
- the number of classes (nc)
- the names of the classes (names)

\
This file is used by YOLO to locate the images and their annotations.

In [ ]:
# YAML dataset (edit routes)
data_yaml = """
train: /content/AERALIS_YOLOv8n_local/train/images
val:   /content/AERALIS_YOLOv8n_local/val/images
test:  /content/AERALIS_YOLOv8n_local/test/images

nc: 1
names: ['person']
"""
open('data.yaml', 'w').write(data_yaml)

176

Perfect, we have correctly written the data.yaml file for the AERALIS_YOLOv8n dataset. Let's continue with the loading of the model:

In [ ]:
# Upload the pre-trained model we want to use as a starting point

model_YOLOv8n = YOLO('yolov8n.pt') # it is the model that will be fine-tuned on the custom dataset

We have now downloaded the pre-trained model from the official Ultralytics repository.

The **Batch size** is the number of images processed simultaneously in each training step. With 4-8GB of RAM, a batch size of 8 (or even less) is recommended, so we'll start with that value and reduce it if necessary.

**Early Stopping** is a technique that automatically stops the training process if the model stops improving after a certain number of epochs. This helps prevent overfitting and saves time.

**Workers** are the parallel processes used to load and preprocess data while the model is training. However, due to our limited resources, we’ll start with 2 workers, and if data loading errors occur, we'll reduce this number to 1 or even 0.


In [ ]:
# To see the available GPU
print(torch.cuda.is_available()) # True = you have GPU --> if False then use device='cpu'
print(torch.cuda.device_count()) # Name of GPU

# If True and at least 1, you can use device=0.
# If you don't have GPU: use device='cpu' (much slower).
# Locally (not Colab): check with nvidia-smi from terminal.

True
1


Now that we have confirmation that the GPU is active we can train the model:

In [ ]:
# Fine‑tuning
results_YOLOv8n = model_YOLOv8n.train(
  data='data.yaml', # use the newly created yaml file
  epochs=100, # Maximum number of training epochs
  imgsz=640, # Image input size (recommended for YOLO).
  batch=16,  # Batch size
  patience=20, # Early stopping if the metrics do not improve for 20 epochs
  workers=2, # Number of workers for the dataloader
  device=0, # Use GPU 0 (or put 'cpu' if you don't have GPU)
  # device='cpu',
  project='runs_finetune', # Folder where it will save the results of the experiments (the folder will be created automatically)
  name='person_yolov8n' # Subfolder/name specific to our experiment
)

# results -->  will contain metrics, logs, and the path of the best weights found during the training

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=person_yolov8n4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12.0, pretrained=True, prof

train: Scanning /content/AERALIS_YOLOv8n_local/train/labels.cache... 2395 images, 388 backgrounds, 0 corrupt: 100%|██████████| 2395/2395 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 901.0±20.7 MB/s, size: 2606.8 KB)


val: Scanning /content/AERALIS_YOLOv8n_local/val/labels.cache... 515 images, 75 backgrounds, 0 corrupt: 100%|██████████| 515/515 [00:00<?, ?it/s]


...

    100/100      2.48G      1.154     0.5461     0.8617         15        640: 100%|██████████| 150/150 [01:04<00:00,  2.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  2.86it/s]

                   all        515       1258      0.896      0.769      0.857      0.481



100 epochs completed in 2.009 hours.
Optimizer stripped from runs_finetune/person_yolov8n4/weights/last.pt, 6.2MB
Optimizer stripped from runs_finetune/person_yolov8n4/weights/best.pt, 6.2MB

Validating runs_finetune/person_yolov8n4/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  2.87it/s]


                   all        515       1258      0.905      0.771      0.861      0.482
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs_finetune/person_yolov8n4


We now want to evaluate the trained model using the Test set defined in data.yaml. \
YOLO does not compute standard accuracy, because in object detection True Negatives (TN) are not counted. Therefore, traditional accuracy is not applicable or useful.


So, we will compute:
- **Precision**: how correct your detected positives are
- **Recall**: how many of the real objects you detected
- **mAP50**: mean Average Precision with IoU ≥ 0.5 (how accurate the predictions are)
- **mAP50-95**: average over various IoU thresholds, a more strict metric
- **F1_score**: combination of precision and recall (you can compute it as: 2 * (P * R) / (P + R))

In [ ]:
# Evaluates the trained model using the TEST SET defined in data.yaml

metrics_YOLOv8n = model_YOLOv8n.val(data='data.yaml', split='test') # returns accuracy metrics (e.g., mAP, precision, recall, etc.) on the test set
print("Test metrics:", metrics_YOLOv8n)

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 80.0±11.5 MB/s, size: 1605.8 KB)


val: Scanning /content/AERALIS_YOLOv8n_local/test/labels... 516 images, 93 backgrounds, 0 corrupt: 100%|██████████| 516/516 [00:01<00:00, 505.99it/s]

val: New cache created: /content/AERALIS_YOLOv8n_local/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:15<00:00,  2.10it/s]


                   all        516       1271      0.905      0.829      0.904      0.506
Speed: 0.3ms preprocess, 1.0ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs_finetune/person_yolov8n42
Test metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d498713b390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031

Confidence (confidence) is the probability estimated by the model that a detected object is actually real (i.e., not a false positive).

In [ ]:
#  Inference (practical use of the fine-tuned model)

# Load the best weights found
best_YOLOv8n = '/content/runs_finetune/person_yolov8n2/weights/best.pt'
model_inf_YOLOv8n = YOLO(best_YOLOv8n) # Creates a new model instance by loading the best weights

# Performs inference on one or more images, or on a video, by specifying the path in the source parameter.
# conf=0.25 → Confidence threshold for considering a detection valid.
preds_YOLOv8n = model_inf_YOLOv8n.predict(
  # source='/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/test/images',
  source='/content/AERALIS_YOLOv8n_local/test/images',
  conf=0.25,
  verbose=False, # disable on-screen printing
  save=True
)

# Too many images to show!
# for result in preds_YOLOv8n:
#    result.show() # displays the predictions (eventually you can also save them)

Results saved to runs/detect/predict


In order to use it on the Jetson Nano we now want to download the model, so:

In [ ]:
# Create folders on Google Drive (first time only):
!mkdir -p /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/results/annotated
!mkdir -p /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/weights

In [ ]:
# Copy annotated images:
!cp -r /content/runs/detect/predict/* /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/results/annotated/

# Copy the best weights (best.pt) after training:
!cp /content/runs_finetune/person_yolov8n2/weights/best.pt /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/weights/

In [ ]:
# To download it directly to the computer:
from google.colab import files
files.download('/content/runs_finetune/person_yolov8n/weights/best.pt')

When we have the .pt file on the PC, we could:
- Upload it to the Jetson Nano,
- Convert it to ONNX/TensorRT if you need it for optimization,
- Use it with the PyTorch/Ultralytics version on any computer.

Weight access code

In [ ]:
# Path of saved weights:
weights_path_YOLOv8n = '/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/weights/best.pt'
model_YOLOv8n = YOLO(weights_path_YOLOv8n)

Access code for annotated images

In [ ]:
# Directory of annotated images
annotated_dir = '/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/results/annotated'

# Get all the .jpg files in the directory
annotated_imgs_YOLOv8n = [
  os.path.join(annotated_dir, f)
  for f in os.listdir(annotated_dir)
  if f.lower().endswith('.jpg')
]

# View the first 5 annotated images
for img_path in annotated_imgs_YOLOv8n[:5]:
  img = Image.open(img_path)
  img.show()

### YOLOv11n

Ora eseguiamo lo stesso procedimento per YOLOv11n:

In [ ]:
src = '/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv11n'
dst = '/content/AERALIS_YOLOv11n_local'  # is now on the local VM, NOT on drive

# If the destination folder already exists, I delete it
if os.path.exists(dst):
  shutil.rmtree(dst)

# Recursive copy of ENTIRE folder (and subfolders)
shutil.copytree(src, dst)
print("Copy completed:", os.path.exists(dst))

Copy completed: True


In [ ]:
!df -h / # It shows the total, used and free space on the root (/) of the Colab VM.

# Avail column: space still available for your files.

Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   55G   58G  49% /


In [ ]:
# Show space used by your local folder
!du -sh /content/AERALIS_YOLOv11n_local

6.6G	/content/AERALIS_YOLOv11n_local


In [ ]:
# Show space occupied by various folders in /content/.
!du -h --max-depth=1 /content/

140K	/content/.config
6.6G	/content/AERALIS_YOLOv11n_local
du: cannot access '/content/drive/.Encrypted/.shortcut-targets-by-id/1LQbD7p_iS5KLqGNdfrYEvsAx0i_bgB0h/projectUPV': No such file or directory
69G	/content/drive
55M	/content/sample_data
75G	/content/


In [ ]:
# YAML dataset (edit routes)
data_yaml = """
train: /content/AERALIS_YOLOv11n_local/train/images
val:   /content/AERALIS_YOLOv11n_local/val/images
test:  /content/AERALIS_YOLOv11n_local/test/images

nc: 1
names: ['person']
"""
open('data.yaml', 'w').write(data_yaml)

179

In [ ]:
# Upload the pre-trained model we want to use as a starting point

model_YOLOv11n = YOLO('yolo11n.pt') # it is the model that will be fine-tuned on the custom dataset

100%|██████████| 5.35M/5.35M [00:00<00:00, 78.4MB/s]


In [ ]:
# To see the available GPU
import torch
print(torch.cuda.is_available()) # True = you have GPU --> if False then use device='cpu'
print(torch.cuda.device_count()) # Name of GPU

# If True and at least 1, you can use device=0.
# If you don't have GPU: use device='cpu' (much slower).
# Locally (not Colab): check with nvidia-smi from terminal.

True
1


In [ ]:
# Fine‑tuning
results_YOLOv11n = model_YOLOv11n.train(
  data='data.yaml',
  epochs=100,
  imgsz=640,
  batch=16,
  patience=20,
  workers=2,
  device=0,
  project='runs_finetune',
  name='person_yolov11n'
)

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=person_yolov11n, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12.0, pretrained=True, prof

100%|██████████| 755k/755k [00:00<00:00, 19.5MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n summary: 181 layers, 2,590,035 parameters, 2,590,019 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2916.3±1744.5 MB/s, size: 2026.4 KB)


train: Scanning /content/AERALIS_YOLOv11n_local/train/labels... 2395 images, 388 backgrounds, 0 corrupt: 100%|██████████| 2395/2395 [00:01<00:00, 1677.09it/s]

...

    100/100         4G       1.22     0.5835     0.8653         15        640: 100%|██████████| 150/150 [01:15<00:00,  2.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:07<00:00,  2.33it/s]

                   all        515       1258      0.877      0.786      0.858      0.479



100 epochs completed in 2.197 hours.
Optimizer stripped from runs_finetune/person_yolov11n/weights/last.pt, 5.5MB
Optimizer stripped from runs_finetune/person_yolov11n/weights/best.pt, 5.5MB

Validating runs_finetune/person_yolov11n/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:07<00:00,  2.32it/s]


                   all        515       1258      0.887      0.778      0.859      0.479
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs_finetune/person_yolov11n


In [ ]:
# Evaluates the trained model using the TEST SET defined in data.yaml

metrics_YOLOv11n = model_YOLOv11n.val(data='data.yaml', split='test') # returns accuracy metrics (e.g., mAP, precision, recall, etc.) on the test set
print("Test metrics:", metrics_YOLOv11n)

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3435.2±1553.1 MB/s, size: 1605.8 KB)


val: Scanning /content/AERALIS_YOLOv11n_local/test/labels... 516 images, 93 backgrounds, 0 corrupt: 100%|██████████| 516/516 [00:00<00:00, 1565.50it/s]

val: New cache created: /content/AERALIS_YOLOv11n_local/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:15<00:00,  2.10it/s]


                   all        516       1271      0.885      0.808      0.882      0.486
Speed: 0.4ms preprocess, 1.6ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs_finetune/person_yolov11n2
Test metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ba49076b190>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031

In [ ]:
#  Inference (practical use of the fine-tuned model)

# Load the best weights found
best_YOLOv11n = '/content/runs_finetune/person_yolov11n/weights/best.pt'
model_inf_YOLOv11n = YOLO(best_YOLOv11n) # Creates a new model instance by loading the best weights

# Performs inference on one or more images, or on a video, by specifying the path in the source parameter.
# conf=0.25 → Confidence threshold for considering a detection valid.
preds_YOLOv11n = model_inf_YOLOv11n.predict(
  # source='/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv11n/test/images',
  source='/content/AERALIS_YOLOv11n_local/test/images',
  conf=0.25,
  verbose=False, # disable on-screen printing
  save=True
)

# Too many images to show!
# for result in preds_YOLOv8n:
#    result.show() # displays the predictions (eventually you can also save them)

Results saved to runs/detect/predict


In [ ]:
# Create folders on Google Drive (first time only):
!mkdir -p /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv11n/results/annotated
!mkdir -p /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv11n/weights

In [ ]:
# Copy annotated images:
!cp -r /content/runs/detect/predict/* /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv11n/results/annotated/

# Copy the best weights (best.pt) after training:
!cp /content/runs_finetune/person_yolov11n/weights/best.pt /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv11n/weights/

In [ ]:
# To download it directly to the computer:
from google.colab import files
files.download('/content/runs_finetune/person_yolov11n/weights/best.pt')

In [ ]:
# Path of saved weights:
weights_path_YOLOv11n = '/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv11n/weights/best.pt'
model_YOLOv11n = YOLO(weights_path_YOLOv11n)

In [ ]:
# Directory of annotated images
annotated_dir = '/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv11n/results/annotated'

# Get all the .jpg files in the directory
annotated_imgs_YOLOv11n = [
    os.path.join(annotated_dir, f)
    for f in os.listdir(annotated_dir)
    if f.lower().endswith('.jpg')
]

# View the first 5 annotated images
for img_path in annotated_imgs_YOLOv11n[:5]:
  img = Image.open(img_path)
  img.show()

### EfficientDet D0

Iniziamo ora ad allenare il modello EfficientDet. Utilizzeremo la libreria _effdet_ di _Ross Wightman_, che semplifica tutto il processo:

In [ ]:
!pip uninstall -y effdet

In [2]:
!pip install git+https://github.com/rwightman/efficientdet-pytorch

  Cloning https://github.com/rwightman/efficientdet-pytorch to /tmp/pip-req-build-ydronsja
  Running command git clone --filter=blob:none --quiet https://github.com/rwightman/efficientdet-pytorch /tmp/pip-req-build-ydronsja
  Resolved https://github.com/rwightman/efficientdet-pytorch to commit c6dff775a36cea0bf9b76c58e59f936411c5ce01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.5 M

In [ ]:
# pip install albumentations[imgaug]

Copy the entire dataset folder from Google Drive to Colab's local storage to increase speed during training:

In [4]:
src = '/content/drive/MyDrive/projectUPV/datasets/AERALIS_EfficientDet_D0'
dst = '/content/AERALIS_EfficientDet_D0_local'

# Delete the destination if it already exists (shutil.rmtree), then recopy from scratch
if os.path.exists(dst):
  shutil.rmtree(dst)
shutil.copytree(src, dst)
print("Copy completed")

Copy completed


In [5]:
!df -h / # It shows the total, used and free space on the root (/) of the Colab VM.

# Avail column: space still available for your files.

Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   56G   58G  50% /


In [6]:
# Show space used by your local folder
!du -sh /content/AERALIS_EfficientDet_D0_local

6.6G	/content/AERALIS_EfficientDet_D0_local


In [7]:
# Show space occupied by various folders in /content/.
!du -h --max-depth=1 /content/

140K	/content/.config
6.6G	/content/AERALIS_EfficientDet_D0_local
du: cannot access '/content/drive/.Encrypted/.shortcut-targets-by-id/1LQbD7p_iS5KLqGNdfrYEvsAx0i_bgB0h/projectUPV': No such file or directory
71G	/content/drive
55M	/content/sample_data
77G	/content/


In [8]:
# Main parameters
BASE_DIR = dst
VARIANT = 'tf_efficientdet_d0'  # Change to 'd1' or 'd2' for other versions
IMAGE_SIZE = 512                # D0=512, D1=640, D2=768
NUM_CLASSES = 1                 # only one class (ex: 'person')
BATCH_SIZE = 16
EPOCHS = 100
PATIENCE = 20
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# To see the available GPU
print(torch.cuda.is_available()) # True = you have GPU --> if False then use device='cpu'
print(torch.cuda.device_count()) # Name of GPU

# If True and at least 1, you can use device=0.
# If you don't have GPU: use device='cpu' (much slower).
# Locally (not Colab): check with nvidia-smi from terminal.

True
1


We keep only those images that contain at least one bounding box:

In [ ]:
class CocoDetectionTransformed(CocoDetection):
  def __init__(self, img_folder, ann_file, transform=None):
    super().__init__(img_folder, ann_file)
    self.transform = transform

    # Build a set of image ids that have annotations with bbox area > 0
    valid_img_ids = set()
    for ann in self.coco.dataset['annotations']:
      if ann['bbox'][2] > 1 and ann['bbox'][3] > 1:  # bbox width & height > 1 pixel
        valid_img_ids.add(ann['image_id'])

    # Find dataset indices corresponding to valid image ids
    self.valid_indices = []
    for i, img_info in enumerate(self.coco.dataset['images']):
      if img_info['id'] in valid_img_ids:
        self.valid_indices.append(i)

  def __len__(self):
    return len(self.valid_indices)

  def __getitem__(self, idx):
    real_idx = self.valid_indices[idx]
    img, target = super().__getitem__(real_idx)
    if self.transform is not None:
      img = self.transform(img)
    return img, target

In [10]:
class CocoDetectionTransformed(CocoDetection):
    def __init__(self, img_folder, ann_file, transform=None):
        super().__init__(img_folder, ann_file)
        self.transform = transform

        # 1) Trovo tutti gli image_id con almeno una bbox >1px
        valid_img_ids = {
            ann['image_id']
            for ann in self.coco.dataset['annotations']
            if ann['bbox'][2] > 1 and ann['bbox'][3] > 1
        }

        # 2) Conservo solo gli indici delle immagini valide
        self.valid_indices = [
            idx for idx, img_info in enumerate(self.coco.dataset['images'])
            if img_info['id'] in valid_img_ids
        ]

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
      real_idx = self.valid_indices[idx]
      img, target = super().__getitem__(real_idx)

      if self.transform:
          bboxes = [obj['bbox'] for obj in target]
          labels = [0] * len(bboxes)
          augmented = self.transform(
              image=np.array(img),
              bboxes=bboxes,
              category_id=labels
          )
          img = augmented['image']
          for obj, new_bbox in zip(target, augmented['bboxes']):
              obj['bbox'] = list(new_bbox)

      # qui estrai l'image_id CORRETTO
      img_id = self.coco.dataset['images'][real_idx]['id']
      return img, target, img_id

Creates a DataLoader to load images and annotations in COCO format:

In [ ]:
# COCO DataLoader Function
def get_loader(img_dir, ann_path, shuffle):
    transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), # Ensures that all images are the same size
        transforms.ToTensor()
    ])
    ds = CocoDetectionTransformed(img_dir, ann_path, transform=transform)
    return DataLoader(ds, batch_size=BATCH_SIZE, shuffle=shuffle,
                      collate_fn=lambda x: tuple(zip(*x)), num_workers=2)

# collate_fn allows for variable object batches (e.g., several boxes per image)

In [11]:
def get_loader(img_dir, ann_path, shuffle):
    # Trasformazioni congiunte immagine + bbox in formato COCO
    transform = A.Compose([
        A.Resize(IMAGE_SIZE, IMAGE_SIZE),
        # A.Normalize(),
        A.Normalize(
          mean=(0.485, 0.456, 0.406),  # ImageNet mean
          std=(0.229, 0.224, 0.225),   # ImageNet std
          max_pixel_value=255.0
        ),
        ToTensorV2(), # converte in torch.Tensor
    ], bbox_params=A.BboxParams(
        format='coco',
        label_fields=['category_id'],
        min_area=1.0,          # scarta bbox con area < 1px
        min_visibility=0.0     # scarta bbox con visibilità < 0%
      ))

    # Passiamo l’albumentations transform al dataset
    ds = CocoDetectionTransformed(img_dir, ann_path, transform=transform)

    return DataLoader(
        ds,
        batch_size=BATCH_SIZE,
        shuffle=shuffle,
        collate_fn=lambda x: tuple(zip(*x)),
        num_workers=2
    )

In [ ]:
# COCO DataLoader Function for inference (con Resize per matchare training)
def get_loader_test(img_dir, ann_path):
  transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor()
  ])
  ds = CocoDetectionTransformed(img_dir, ann_path, transform=transform)
  return DataLoader(ds, batch_size=1, shuffle=False,
                    collate_fn=lambda x: tuple(zip(*x)), num_workers=2)

In [12]:
def get_loader_test(img_dir, ann_path):
    transform = A.Compose([
        A.Resize(IMAGE_SIZE, IMAGE_SIZE),
        # A.Normalize(),
        A.Normalize(
          mean=(0.485, 0.456, 0.406),  # ImageNet mean
          std=(0.229, 0.224, 0.225),   # ImageNet std
          max_pixel_value=255.0
        ),
        ToTensorV2(),
    ], bbox_params=A.BboxParams(
        format='coco',
        label_fields=['category_id'],
        min_area=1.0,          # scarta bbox con area < 1px
        min_visibility=0.0     # scarta bbox con visibilità < 0%
      ))

    ds = CocoDetectionTransformed(img_dir, ann_path, transform=transform)

    return DataLoader(
        ds,
        batch_size=1,
        shuffle=False,
        collate_fn=lambda x: tuple(zip(*x)),
        num_workers=2
    )

In [13]:
# Loading the 3 sets (train, val, test):
train_loader = get_loader(f'{BASE_DIR}/train/images', f'{BASE_DIR}/annotations_train.json', True)
val_loader   = get_loader(f'{BASE_DIR}/val/images', f'{BASE_DIR}/annotations_val.json', False)
test_loader = get_loader_test(f'{BASE_DIR}/test/images', f'{BASE_DIR}/annotations_test.json')

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Adam (short for Adaptive Moment Estimation) is one of the most widely used optimizers in deep learning.

The optimizer is an algorithm that updates model weights during training to reduce loss.
- _AdamW_ is a variant of Adam, which also includes a weight penalty (weight decay) in a more correct way than classical Adam.
- The _W_ stands for “Weight Decay fix”: it improves regularization and reduces overfitting.

In this way:
- Converges quickly (like Adam)
- Reduces overfitting better than Adam
- Is the standard in many modern models (such as BERT, EfficientDet, etc.).

In [14]:
# Model initialization
config = get_efficientdet_config(VARIANT) # load configuration for tf_efficientdet_d0
config.num_classes = NUM_CLASSES # sets the number of classes
config.image_size = (IMAGE_SIZE, IMAGE_SIZE) # sets the image size
# config.image_size = None # This tells the model to accept input of any size and apply the correct resizing internally
net = EfficientDet(config, pretrained_backbone=True) # Initialize the model (use pre-trained backbone)
# net = EfficientDet(config)
net.class_net = HeadNet(config, num_outputs=NUM_CLASSES) # Create the classifier (HeadNet)
model = DetBenchTrain(net, config).to(DEVICE) # He wraps it in a training module

# CONGELAMENTO del backbone
# Blocca i gradenti del backbone: nelle prime epoche si addestrano solo class_net e box_net
for param in net.backbone.parameters():
    param.requires_grad = False  # freeze backbone layers :contentReference[oaicite:3]{index=3}

# Ottimizzatore con solo parametri “trainabili”
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4
)  # learning rate iniziale per testate :contentReference[oaicite:4]{index=4}
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4) # Use AdamW optimizer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

HeadNet is the final block of the EfficientDet model that does box classification and regression.

EfficientDet has two “heads”:
- Classification head: says “what” is in the image (object class)
- Regression head: says “where” it is (bounding box)

When we use: _net.class_net = HeadNet(config, num_outputs=NUM_CLASSES)_ we are customizing the classification head of the model to use our number of classes, for example 1 for ‘person’.

Without this change, the model would remain pre-configured for COCO (80 classes), so it would miss everything in the custom dataset.

Now initialize values to track the best validation and apply early stopping:

In [18]:
# Numero di epoche dopo cui “scongelare” il backbone
freeze_epochs = 10

# Early stopping setup:
epoch = 0
epochs_no_improve = 0  # Count how many epochs passed without validation improvement
best_loss = float('inf')  # Best (minimal) validation loss so far
best_model_path = f'effdet_{VARIANT}_best.pth'  # File to save best weights

# Training loop
while epoch < EPOCHS and epochs_no_improve < PATIENCE:
    if epoch == freeze_epochs:
        for param in net.backbone.parameters():
            param.requires_grad = True   # ← unfreeze backbone layers
        # ricrea l’optimizer per includere ora tutti i parametri
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
        print(f"[Info] Backbone scongelato all'epoca {epoch}")

    model.train()  # Training mode
    total_loss = 0.0  # Sum of training losses this epoch

    for images, targets, _ in train_loader: # con _ scarto 'silenziosamente' il terzo elemento quando non serve
        images = torch.stack([img.to(DEVICE) for img in images])

        boxes = []
        labels = []

        for t in targets:
            b = []
            l = []
            for obj in t:
                x, y, w, h = obj['bbox']
                if w > 1 and h > 1:  # Filter invalid bboxes
                    x1, y1 = x, y
                    x2, y2 = x + w, y + h
                    b.append([x1, y1, x2, y2])
                    l.append(0)  # Class index (0 for single class)

            b = torch.tensor(b, dtype=torch.float32)
            l = torch.tensor(l, dtype=torch.int64)
            boxes.append(b.to(DEVICE))
            labels.append(l.to(DEVICE))

        # img_sizes = torch.tensor([[IMAGE_SIZE, IMAGE_SIZE]] * len(images), device=DEVICE)
        # img_sizes = torch.tensor([[img.shape[1], img.shape[2]] for img in images], device=DEVICE)
        # img_scales = torch.ones(len(images), device=DEVICE)
        img_sizes  = torch.tensor([[img.shape[1], img.shape[2]] for img in images], device=DEVICE)
        img_scales = torch.ones(len(images), device=DEVICE)

        loss_dict = model(images, {
            'bbox': boxes,
            'cls':  labels,
            'img_scale': img_scales,
            'img_size': img_sizes
        })

        # Forward pass and loss calculation
        #loss_dict = model(images, {
          #   'bbox': boxes,
          #  'cls': labels,
            # 'img_scale': img_scales,
            #'img_size': img_sizes
        #})
        loss = loss_dict['loss']

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        print(f"[Epoch {epoch}] Batch loss: {loss.item()}")

    # Validation phase
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for images, targets, _ in val_loader:
            images = torch.stack([img.to(DEVICE) for img in images])

            boxes = []
            labels = []

            for t in targets:
                b = []
                l = []
                for obj in t:
                    x, y, w, h = obj['bbox']
                    if w > 1 and h > 1:
                        x1, y1 = x, y
                        x2, y2 = x + w, y + h
                        b.append([x1, y1, x2, y2])
                        l.append(0)
                b = torch.tensor(b, dtype=torch.float32)
                l = torch.tensor(l, dtype=torch.int64)
                boxes.append(b.to(DEVICE))
                labels.append(l.to(DEVICE))

            # img_sizes = torch.tensor([[IMAGE_SIZE, IMAGE_SIZE]] * len(images), device=DEVICE)
            # img_sizes = torch.tensor([[img.shape[1], img.shape[2]] for img in images], device=DEVICE)
            # img_scales = torch.ones(len(images), device=DEVICE)

            img_sizes  = torch.tensor([[img.shape[1], img.shape[2]] for img in images], device=DEVICE)
            img_scales = torch.ones(len(images), device=DEVICE)

            loss_dict = model(images, {
                'bbox': boxes,
                'cls':  labels,
                'img_scale': img_scales,
                'img_size': img_sizes
            })
            #loss_dict = model(images, {
             #   'bbox': boxes,
              #  'cls': labels,
               # 'img_scale': img_scales,
                #'img_size': img_sizes
            #})
            loss = loss_dict['loss']

            if not torch.isnan(loss):
                val_loss += loss.item()
            else:
                print("NaN detected in loss, batch skipped.")

    avg_val_loss = val_loss / len(val_loader)
    print(f"[Epoch {epoch+1}/{EPOCHS}] Val loss: {avg_val_loss:.4f}")

    # Early stopping check
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        epochs_no_improve = 0
        print("Best saved model.")
    else:
        epochs_no_improve += 1
        print(f"Epochs without improvement: {epochs_no_improve}/{PATIENCE}")

    epoch += 1

# Training end reason
if epochs_no_improve >= PATIENCE:
    print("Early stopping enabled.")
else:
    print("End of training: reached maximum number of epochs.")

Streaming output truncated to the last 5000 lines.
[Epoch 60] Batch loss: 0.5305060744285583
[Epoch 60] Batch loss: 0.7975022792816162
[Epoch 60] Batch loss: 0.7384965419769287
[Epoch 60] Batch loss: 0.7670215368270874
[Epoch 60] Batch loss: 0.8271852731704712
[Epoch 61/100] Val loss: 1.1332
Best saved model.
[Epoch 61] Batch loss: 0.9066474437713623
[Epoch 61] Batch loss: 0.6576187610626221
[Epoch 61] Batch loss: 1.01454496383667
[Epoch 61] Batch loss: 0.7341536283493042
[Epoch 61] Batch loss: 0.5500481724739075
[Epoch 61] Batch loss: 0.7822741270065308
[Epoch 61] Batch loss: 0.7279778122901917
[Epoch 61] Batch loss: 0.6833098530769348
[Epoch 61] Batch loss: 0.5590689778327942
[Epoch 61] Batch loss: 0.5873044729232788
[Epoch 61] Batch loss: 0.6472947597503662
[Epoch 61] Batch loss: 0.5579423308372498
[Epoch 61] Batch loss: 0.5199979543685913
[Epoch 61] Batch loss: 0.5153982043266296
[Epoch 61] Batch loss: 0.6891880035400391
[Epoch 61] Batch loss: 0.6157970428466797
[Epoch 61] Batch lo

Copy the weights of the best saved model (.pth) from your local file system to your Google Drive folder:

In [19]:
!mkdir -p /content/drive/MyDrive/projectUPV/datasets/AERALIS_EfficientDet_D0/weights

Verifichiamo che il file esista:

In [20]:
if not os.path.isfile(best_model_path):
  raise FileNotFoundError(f"Il file dei migliori pesi non è stato trovato: {best_model_path}")

weights_dir = '/content/drive/MyDrive/projectUPV/datasets/AERALIS_EfficientDet_D0/weights'
os.makedirs(weights_dir, exist_ok=True)
shutil.copy(best_model_path, weights_dir)
print("EfficientDet weights copied to Google Drive.")

EfficientDet weights copied to Google Drive.


Load the model in evaluation mode (inference):

In [21]:
config = get_efficientdet_config(VARIANT)
config.num_classes = NUM_CLASSES
config.image_size = (IMAGE_SIZE, IMAGE_SIZE)

net = EfficientDet(config, pretrained_backbone=False)
net.class_net = HeadNet(config, num_outputs=NUM_CLASSES)

model = DetBenchPredict(net).to(DEVICE)
model.load_state_dict(torch.load(best_model_path))
model.eval()

DetBenchPredict(
  (model): EfficientDet(
    (backbone): EfficientNetFeatures(
      (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn1): BatchNormAct2d(
        32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): SiLU(inplace=True)
      )
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (bn1): BatchNormAct2d(
              32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): SiLU(inplace=True)
            )
            (aa): Identity()
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (act1): SiLU(inplace=True)
              (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1

In [24]:
# Inference on complete test set
predictions = []

with torch.no_grad():
    for imgs, targets, img_ids in test_loader:
        for img, target, img_id in zip(imgs, targets, img_ids):
            img_tensor = img.unsqueeze(0).to(DEVICE)
            outputs = model(img_tensor)
            output = outputs[0]
            if output.ndim != 2 or output.shape[1] != 6:
                continue

            boxes  = output[:, :4].cpu().numpy()
            scores = output[:, 4].cpu().numpy()
            labels = output[:, 5].cpu().numpy().astype(int)

            # dimensioni originali
            orig_w = test_loader.dataset.coco.imgs[img_id]['width']
            orig_h = test_loader.dataset.coco.imgs[img_id]['height']
            scale_x = orig_w / IMAGE_SIZE
            scale_y = orig_h / IMAGE_SIZE

            for box, score, label in zip(boxes, scores, labels):
                x1, y1, x2, y2 = box

                # 1) ripristino dimensioni e clamp
                x1 = max(0,     x1 * scale_x)
                y1 = max(0,     y1 * scale_y)
                x2 = min(orig_w, x2 * scale_x)
                y2 = min(orig_h, y2 * scale_y)

                # 2) calcolo larghezza/altezza minime
                w = max(1, x2 - x1)
                h = max(1, y2 - y1)

                # 3) appendo predizione “pulita”
                predictions.append({
                    "image_id":    img_id,
                    "category_id": label + 1,
                    "bbox":        [x1, y1, w, h],
                    "score":       float(score)
                })

print(f"\nInference completed. Total predictions collected: {len(predictions)}")


Inference completed. Total predictions collected: 42300


We save all predictions in a .json file in the format required by COCOeval:

In [25]:
# Convert all fields to native Python types
for pred in predictions:
    pred["category_id"] = int(pred["category_id"])
    pred["image_id"] = int(pred["image_id"])
    pred["score"] = float(pred["score"])
    pred["bbox"] = [float(x) for x in pred["bbox"]]

# Save to COCO-style JSON
with open("results_coco.json", "w") as f:
    json.dump(predictions, f)

Final evaluation with pycocotools:

In [29]:
# Load ground truth annotations
coco_gt = COCO(f"{BASE_DIR}/annotations_test.json")

# Assicuriamoci che esista il campo 'info'
if 'info' not in coco_gt.dataset:
    coco_gt.dataset['info'] = {}

# Load predictions from file …
with open("results_coco.json", "r") as f:
    results = json.load(f)

# … poi prosegui con loadRes e COCOeval
coco_dt = coco_gt.loadRes(results)
coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.06s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

We note that values of AP and AR practically at zero indicate that the model is not making correct or meaningful predictions. In essence, it is not recognizing objects well. \
Let's try a small test on a few images and annotations to see if the model recognizes anything.

At this point, it may be worth considering that EfficientDet-D0 (or your current EfficientDet configuration) might not be the ideal detector for your specific dataset.

In addition to changing the model, we could also:
- Review preprocessing and augmentations to make the training data more representative of the test set.
- Experiment with learning rate schedulers (e.g., ReduceLROnPlateau or CosineAnnealingLR) to improve convergence.
- Adjust inference filtering thresholds (score and minimum box size) so as not to discard potentially valid predictions.

### MobileNetV2 + SSDLite

### MobileNetV3 + SSDLite